In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import albumentations as A
import gc
import matplotlib.pyplot as plt
import math
import multiprocessing
import numpy as np
import os
import pandas as pd
import random
import time
import timm
import torch
import torch.nn as nn


from albumentations.pytorch import ToTensorV2
from glob import glob
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from typing import Dict, List

os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

# CPMP: avoids some issues when using more than one worker
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', torch.cuda.device_count(), 'GPU(s)')
device

Using 1 GPU(s)


device(type='cuda')

In [2]:
class config:
    AMP = True
    BATCH_SIZE_TRAIN = 100
    BATCH_SIZE_VALID = 100
    EPOCHS = 12
    FOLDS = 6
    FREEZE = False
    GRADIENT_ACCUMULATION_STEPS = 1
    MAX_GRAD_NORM = 1e7
    MODEL = "regnetx_064"
    NUM_FROZEN_LAYERS = 20
    NUM_WORKERS = 4 # multiprocessing.cpu_count()
    PRINT_FREQ = 20
    SEED = 20
    TRAIN_FULL_DATA = False
    VISUALIZE = True
    WEIGHT_DECAY = 0.02
    
    
class paths:
    OUTPUT_DIR = "/kaggle/working/"
    PRE_LOADED_EEGS = '/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy'
    PRE_LOADED_SPECTOGRAMS = '/kaggle/input/brain-spectrograms/specs.npy'
    TRAIN_CSV = "/kaggle/input/hms-harmful-brain-activity-classification/train.csv"
    TRAIN_EEGS = "/kaggle/input/brain-eeg-spectrograms/EEG_Spectrograms/"
    TRAIN_SPECTOGRAMS = "/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/"

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s: float):
    "Convert to minutes."
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since: float, percent: float):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def get_logger(filename=paths.OUTPUT_DIR):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def plot_spectrogram(spectrogram_path: str):
    """
    Source: https://www.kaggle.com/code/mvvppp/hms-eda-and-domain-journey
    Visualize spectogram recordings from a parquet file.
    :param spectrogram_path: path to the spectogram parquet.
    """
    sample_spect = pd.read_parquet(spectrogram_path)
    
    split_spect = {
        "LL": sample_spect.filter(regex='^LL', axis=1),
        "RL": sample_spect.filter(regex='^RL', axis=1),
        "RP": sample_spect.filter(regex='^RP', axis=1),
        "LP": sample_spect.filter(regex='^LP', axis=1),
    }
    
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))
    axes = axes.flatten()
    label_interval = 5
    for i, split_name in enumerate(split_spect.keys()):
        ax = axes[i]
        img = ax.imshow(np.log(split_spect[split_name]).T, cmap='viridis', aspect='auto', origin='lower')
        cbar = fig.colorbar(img, ax=ax)
        cbar.set_label('Log(Value)')
        ax.set_title(split_name)
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time")

        ax.set_yticks(np.arange(len(split_spect[split_name].columns)))
        ax.set_yticklabels([column_name[3:] for column_name in split_spect[split_name].columns])
        frequencies = [column_name[3:] for column_name in split_spect[split_name].columns]
        ax.set_yticks(np.arange(0, len(split_spect[split_name].columns), label_interval))
        ax.set_yticklabels(frequencies[::label_interval])
    plt.tight_layout()
    plt.show()
    
    
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 

    
def sep():
    print("-"*100)
    

target_preds = [x + "_pred" for x in ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']]
label_to_num = {'Seizure': 0, 'LPD': 1, 'GPD': 2, 'LRDA': 3, 'GRDA': 4, 'Other':5}
num_to_label = {v: k for k, v in label_to_num.items()}
LOGGER = get_logger()
seed_everything(config.SEED)

In [4]:
df = pd.read_csv(paths.TRAIN_CSV)
label_cols = df.columns[-6:]
print(f"Train cataframe shape is: {df.shape}")
print(f"Labels: {list(label_cols)}")
df.head()

Train cataframe shape is: (106800, 15)
Labels: ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [5]:
train_df = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg({
    'spectrogram_id':'first',
    'spectrogram_label_offset_seconds':'min'
})
train_df.columns = ['spectogram_id','min']

aux = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg({
    'spectrogram_label_offset_seconds':'max'
})
train_df['max'] = aux

aux = df.groupby('eeg_id')[['patient_id']].agg('first')
train_df['patient_id'] = aux

aux = df.groupby('eeg_id')[label_cols].agg('sum')
for label in label_cols:
    train_df[label] = aux[label].values
    
y_data = train_df[label_cols].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train_df[label_cols] = y_data

aux = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train_df['target'] = aux

train_df = train_df.reset_index()
print('Train non-overlapp eeg_id shape:', train_df.shape )
train_df.head()

Train non-overlapp eeg_id shape: (17089, 12)


,eeg_id,spectogram_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [6]:
%%time
READ_SPEC_FILES = False

paths_spectograms = glob(paths.TRAIN_SPECTOGRAMS + "*.parquet")
print(f'There are {len(paths_spectograms)} spectrogram parquets')

if READ_SPEC_FILES:    
    all_spectrograms = {}
    for file_path in tqdm(paths_spectograms):
        aux = pd.read_parquet(file_path)
        name = int(file_path.split("/")[-1].split('.')[0])
        all_spectrograms[name] = aux.iloc[:,1:].values
        del aux
else:
    all_spectrograms = np.load(paths.PRE_LOADED_SPECTOGRAMS, allow_pickle=True).item()
    
if config.VISUALIZE:
    idx = np.random.randint(0,len(paths_spectograms))
    spectrogram_path = paths_spectograms[idx]
    plot_spectrogram(spectrogram_path)

There are 11138 spectrogram parquets


CPU times: user 6.24 s, sys: 6.63 s, total: 12.9 s
Wall time: 1min 6s


In [ ]:
%%time
READ_EEG_SPEC_FILES = False

paths_eegs = glob(paths.TRAIN_EEGS + "*.npy")
print(f'There are {len(paths_eegs)} EEG spectograms')

if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for file_path in tqdm(paths_eegs):
        eeg_id = file_path.split("/")[-1].split(".")[0]
        eeg_spectogram = np.load(file_path)
        all_eegs[eeg_id] = eeg_spectogram
else:
    all_eegs = np.load(paths.PRE_LOADED_EEGS, allow_pickle=True).item()

There are 17089 EEG spectograms


In [ ]:
from sklearn.model_selection import KFold, GroupKFold


gkf = GroupKFold(n_splits=config.FOLDS)
for fold, (train_index, valid_index) in enumerate(gkf.split(train_df, train_df.target, train_df.patient_id)):
    train_df.loc[valid_index, "fold"] = int(fold)
    
display(train_df.groupby('fold').size()), sep()
display(train_df.head())

In [ ]:
class CustomDataset(Dataset):
    def __init__(
        self, df: pd.DataFrame, config,
        augment: bool = False, mode: str = 'train',
        specs: Dict[int, np.ndarray] = all_spectrograms,
        eeg_specs: Dict[int, np.ndarray] = all_eegs
    ): 
        self.df = df
        self.config = config
        self.batch_size = self.config.BATCH_SIZE_TRAIN
        self.augment = augment
        self.mode = mode
        self.spectograms = all_spectrograms
        self.eeg_spectograms = eeg_specs
        
    def __len__(self):
        """
        Denotes the number of batches per epoch.
        """
        return len(self.df)
        
    def __getitem__(self, index):
        """
        Generate one batch of data.
        """
        X, y = self.__data_generation(index)
        if self.augment:
            X = self.__transform(X) 
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
                        
    def __data_generation(self, index):
        """
        Generates data containing batch_size samples.
        """
        X = np.zeros((128, 256, 8), dtype='float32')
        y = np.zeros(6, dtype='float32')
        img = np.ones((128,256), dtype='float32')
        row = self.df.iloc[index]
        if self.mode=='test': 
            r = 0
        else: 
            r = int((row['min'] + row['max']) // 4)
            
        for region in range(4):
            img = self.spectograms[row.spectogram_id][r:r+300, region*100:(region+1)*100].T
            
            # Log transform spectogram
            img = np.clip(img, np.exp(-4), np.exp(8))
            img = np.log(img)

            # Standarize per image
            ep = 1e-8
            xmax = np.nanmax(img.flatten())
            xmin = np.nanmin(img.flatten())
            img = (img-xmin)/(xmax-xmin+ep)
            #img = (img-mu)/(std+ep)
            img = np.nan_to_num(img, nan=0.0)
            X[14:-14, :, region] = img[:, 22:-22] / 2.0
            img = self.eeg_spectograms[row.eeg_id]
            X[:, :, 4:] = img
#             print(mu)
#             print(std)
            if self.mode != 'test':
                y = row[label_cols].values.astype(np.float32)
            
        return X, y
    
    def __transform(self, img):
        transforms = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.CoarseDropout(max_holes=4,max_height=54,max_width=18,fill_value=0,p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.AdvancedBlur(p=.5),
            #A.ChannelDropout((1,1), p=.5), 
            #A.transforms.ChannelShuffle(),
            A.transforms.PixelDropout(dropout_prob=0.01,p=.5), 
            #A.transforms.Sharpen() 
        ])
        return transforms(image=img)['image']

In [ ]:
train_dataset = CustomDataset(train_df, config, mode="train", augment=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE_TRAIN,
    shuffle=True,
    num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
)
X, y = train_dataset[0]
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
plt.imshow(X[:,:,7])


In [ ]:
timm.list_models()

In [ ]:
class EEGEffnetB0(nn.Module):
    
    def __init__(self, config, num_classes: int = 6, pretrained: bool = True):
        super(EEGEffnetB0, self).__init__()
        self.fc0 = nn.Linear(8, 3)
        #self.fc01 = nn.Linear(64, 16)
        #self.fc02 = nn.Linear(16, 3)
        self.timm_model = timm.create_model('regnetx_064',
            #config.MODEL,
            pretrained=pretrained,
            drop_rate = 0.2,
            drop_path_rate = 0.2,
            num_classes=6
        )
        #self.features = nn.Sequential(*list(self.timm_model.children())[:-2])
#         self.fc4 = nn.Linear(4, 1024)
#         self.fc41 = nn.Linear(1024, 512)
#         self.fc42 = nn.Linear(512, 1)
        
        self.fc20 = nn.Linear(43520, 400)
        #self.fc21 = nn.Linear(8000, 4000)
        #self.fc22 = nn.Linear(4000, 2000)

#         self.fc1 = nn.Linear(8, 400)
        #self.fc2 = nn.Linear(2000 , 400)
#         
        self.fc3 = nn.Linear(400, num_classes)
        
    def forward(self, x):
        #x = x[:, :, :, 4:]
        b = x.shape[0]
        x = F.relu(self.fc0(x))
        #x = F.gelu(self.fc01(x))
        #x = F.gelu(self.fc02(x))
        x = x.permute(0,3,2,1)
        x = F.relu(self.timm_model(x))
        x = x.view(b, -1)
        #x = x.squeeze()
#         x = F.gelu(self.fc4(x))
#         x = F.gelu(self.fc41(x))
#         x = F.gelu(self.fc42(x))
#         x = x.squeeze()
#         x = x.permute(0,2,1)
        
        x = F.relu(self.fc20(x))
      #  x = F.gelu(self.fc21(x))
      #  x = F.gelu(self.fc22(x))
#         x = x.squeeze()
        
#         x = F.gelu(self.fc1(x))
       # x = F.relu(self.fc2(x))

        #print(x.shape)
        out = self.fc3(x)
        return F.log_softmax(out, dim=1)


class CustomModel(nn.Module):
    def __init__(self, config, num_classes: int = 6, pretrained: bool = True):
        super(CustomModel, self).__init__()
        self.USE_KAGGLE_SPECTROGRAMS = False
        self.USE_EEG_SPECTROGRAMS = True
        self.model = timm.create_model(
            config.MODEL,
            pretrained=pretrained,
            drop_rate = 0.1,
            drop_path_rate = 0.2,
        )
        if config.FREEZE:
            for i,(name, param) in enumerate(list(self.model.named_parameters())\
                                             [0:config.NUM_FROZEN_LAYERS]):
                param.requires_grad = False

        self.features = nn.Sequential(*list(self.model.children())[:-2])
        self.custom_layers = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(self.model.num_features, num_classes)
        )

    def __reshape_input(self, x):
        """
        Reshapes input (128, 256, 8) -> (512, 512, 3) monotone image.
        """ 
        # === Get spectograms ===
        spectograms = [x[:, :, :, i:i+1] for i in range(4)]
        spectograms = torch.cat(spectograms, dim=1)
        
        # === Get EEG spectograms ===
        eegs = [x[:, :, :, i:i+1] for i in range(4,8)]
        eegs = torch.cat(eegs, dim=1)
        
        # === Reshape (512,512,3) ===
        if self.USE_KAGGLE_SPECTROGRAMS & self.USE_EEG_SPECTROGRAMS:
            x = torch.cat([spectograms, eegs], dim=2)
        elif self.USE_EEG_SPECTROGRAMS:
            x = eegs
        else:
            x = spectograms
            
        x = torch.cat([x,x,x], dim=3)
        x = x.permute(0, 3, 1, 2)
        return x
    
    def forward(self, x):
        x = self.__reshape_input(x)
        x = self.features(x)
        x = self.custom_layers(x)
        return x

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR

# EPOCHS = config.EPOCHS
# BATCHES = len(train_loader)
# steps = []
# lrs = []
# optim_lrs = []
# model = EEGEffnetB0()
# model=torch.nn.DataParallel(model, device_ids=[0, 1])
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# scheduler = OneCycleLR(
#     optimizer,
#     max_lr=1e-3,
#     epochs=config.EPOCHS,
#     steps_per_epoch=len(train_loader),
#     pct_start=0.05,
#     anneal_strategy="cos",
#     final_div_factor=100,
# )
# for epoch in range(EPOCHS):
#     for batch in range(BATCHES):
#         scheduler.step()
#         lrs.append(scheduler.get_last_lr()[0])
#         steps.append(epoch * BATCHES + batch)

# max_lr = max(lrs)
# min_lr = min(lrs)
# print(f"Maximum LR: {max_lr} | Minimum LR: {min_lr}")
# plt.figure()
# plt.plot(steps, lrs, label='OneCycle')
# plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
# plt.xlabel("Step")
# plt.ylabel("Learning Rate")
# plt.show()

In [ ]:
import torch.nn.functional as F

# === Reduction = "mean" ===
# criterion = nn.KLDivLoss(reduction="mean")
# y_pred = F.log_softmax(torch.randn(6, 2, requires_grad=True), dim=1)
# y_true = F.softmax(torch.rand(6, 2), dim=1)
# print(f"Predictions: {y_pred}")
# print(f"Targets: {y_true}")
# output = criterion(y_pred, y_true)
# print(f"Output: {output}")

# print("\n", "="*100, "\n")

# # === Reduction = "batchmean" ===
# criterion = nn.KLDivLoss(reduction="batchmean")
# y_pred = F.log_softmax(torch.randn(2, 6, requires_grad=True), dim=1)
# y_true = F.softmax(torch.rand(2, 6), dim=1)
# print(f"Predictions: {y_pred}")
# print(f"Targets: {y_true}")
# output = criterion(y_pred, y_true)
# print(f"Output: {output}")

In [ ]:
def train_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    """One epoch training pass."""
    model.train() 
    criterion = nn.KLDivLoss(reduction="batchmean")
    scaler = torch.cuda.amp.GradScaler(enabled=config.AMP)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    
    # ========== ITERATE OVER TRAIN BATCHES ============
    with tqdm(train_loader, unit="train_batch", desc='Train') as tqdm_train_loader:
        for step, (X, y) in enumerate(tqdm_train_loader):
            X = X.to(device)
            y = y.to(device)
            batch_size = y.size(0)
            with torch.cuda.amp.autocast(enabled=config.AMP):
                y_preds = model(X) 
                loss = criterion(F.log_softmax(y_preds, dim=1), y)
            if config.GRADIENT_ACCUMULATION_STEPS > 1:
                loss = loss / config.GRADIENT_ACCUMULATION_STEPS
            losses.update(loss.item(), batch_size)
            scaler.scale(loss).backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)

            if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1
                scheduler.step()
            end = time.time()

            # ========== LOG INFO ==========
            if step % config.PRINT_FREQ == 0 or step == (len(train_loader)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      'Elapsed {remain:s} '
                      'Loss: {loss.avg:.4f} '
                      'Grad: {grad_norm:.4f}  '
                      'LR: {lr:.8f}  '
                      .format(epoch+1, step, len(train_loader), 
                              remain=timeSince(start, float(step+1)/len(train_loader)),
                              loss=losses,
                              grad_norm=grad_norm,
                              lr=scheduler.get_last_lr()[0]))

    return losses.avg


def valid_epoch(valid_loader, model, criterion, device):
    model.eval()
    softmax = nn.Softmax(dim=1)
    losses = AverageMeter()
    prediction_dict = {}
    preds = []
    start = end = time.time()
    with tqdm(valid_loader, unit="valid_batch", desc='Validation') as tqdm_valid_loader:
        for step, (X, y) in enumerate(tqdm_valid_loader):
            X = X.to(device)
            y = y.to(device)
            batch_size = y.size(0)
            with torch.no_grad():
                y_preds = model(X)
                loss = criterion(F.log_softmax(y_preds, dim=1), y)
            if config.GRADIENT_ACCUMULATION_STEPS > 1:
                loss = loss / config.GRADIENT_ACCUMULATION_STEPS
            losses.update(loss.item(), batch_size)
            y_preds = softmax(y_preds)
            preds.append(y_preds.to('cpu').numpy())
            end = time.time()

            # ========== LOG INFO ==========
            if step % config.PRINT_FREQ == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                      'Elapsed {remain:s} '
                      'Loss: {loss.avg:.4f} '
                      .format(step, len(valid_loader),
                              remain=timeSince(start, float(step+1)/len(valid_loader)),
                              loss=losses))
                
    prediction_dict["predictions"] = np.concatenate(preds)
    return losses.avg, prediction_dict

In [ ]:
def train_loop(df, fold):
    
    LOGGER.info(f"========== Fold: {fold} training ==========")

    # ======== SPLIT ==========
    train_folds = df[df['fold'] != fold].reset_index(drop=True)
    valid_folds = df[df['fold'] == fold].reset_index(drop=True)
    
    # ======== DATASETS ==========
    train_dataset = CustomDataset(train_folds, config, mode="train", augment=True)
    valid_dataset = CustomDataset(valid_folds, config, mode="train", augment=False)
    
    # ======== DATALOADERS ==========
    train_loader = DataLoader(train_dataset,
                              batch_size=config.BATCH_SIZE_TRAIN,
                              shuffle=False,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=config.BATCH_SIZE_VALID,
                              shuffle=False,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False)
    
    # ======== MODEL ==========
    model = EEGEffnetB0(config)
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=config.WEIGHT_DECAY)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=4e-4,
        epochs=config.EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.05,
        anneal_strategy="cos",
        final_div_factor=100,
    )

    # ======= LOSS ==========
    criterion = nn.KLDivLoss(reduction="batchmean")
    
    best_loss = np.inf
    # ====== ITERATE EPOCHS ========
    for epoch in range(config.EPOCHS):
        start_time = time.time()

        # ======= TRAIN ==========
        avg_train_loss = train_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # ======= EVALUATION ==========
        avg_val_loss, prediction_dict = valid_epoch(valid_loader, model, criterion, device)
        predictions = prediction_dict["predictions"]
        
        # ======= SCORING ==========
        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        paths.OUTPUT_DIR + f"/{config.MODEL.replace('/', '_')}_fold_{fold}_best.pth")

    predictions = torch.load(paths.OUTPUT_DIR + f"/{config.MODEL.replace('/', '_')}_fold_{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[target_preds] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
def train_loop_full_data(df):
    train_dataset = CustomDataset(df, config, mode="train", augment=True)
    train_loader = DataLoader(train_dataset,
                              batch_size=config.BATCH_SIZE_TRAIN,
                              shuffle=False,
                              num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True)
    model = CustomModel(config)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=config.WEIGHT_DECAY)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=config.EPOCHS,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy="cos",
        final_div_factor=100,
    )
    criterion = nn.KLDivLoss(reduction="batchmean")
    best_loss = np.inf
    for epoch in range(config.EPOCHS):
        start_time = time.time()
        avg_train_loss = train_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device)
        elapsed = time.time() - start_time
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_train_loss:.4f}  time: {elapsed:.0f}s')
        torch.save(
            {'model': model.state_dict()},
            paths.OUTPUT_DIR + f"/{config.MODEL.replace('/', '_')}_epoch_{epoch}.pth")
    torch.cuda.empty_cache()
    gc.collect()
    return _

In [ ]:
def get_result(oof_df):
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    labels = torch.tensor(oof_df[label_cols].values)
    preds = torch.tensor(oof_df[target_preds].values)
    preds = F.log_softmax(preds, dim=1)
    result = kl_loss(preds, labels)
    return result

if not config.TRAIN_FULL_DATA:
    oof_df = pd.DataFrame()
    for fold in range(config.FOLDS):
        if fold in [0, 1, 2, 3, 4,5]:
            _oof_df = train_loop(train_df, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== Fold {fold} result: {get_result(_oof_df)} ==========")
            print(f"========== Fold {fold} result: {get_result(_oof_df)} ==========")
    oof_df = oof_df.reset_index(drop=True)
    LOGGER.info(f"========== CV: {get_result(oof_df)} ==========")
    oof_df.to_csv(paths.OUTPUT_DIR + '/oof_df.csv', index=False)
else:
    train_loop_full_data(train_df)

In [ ]:
model2 = EEGEffnetB0(config)
model_weight = '/kaggle/input/hms-effb0-model0/tf_efficientnet_b0_fold_0_best.pth'
checkpoint = torch.load(model_weight)
model2.load_state_dict(checkpoint["model"])
model2.to(device)


In [ ]:
# import timm
# from timm import create_model
# import pandas as pd
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# VER = 5
# # IF THIS EQUALS NONE, THEN WE TRAIN NEW MODELS
# # IF THIS EQUALS DISK PATH, THEN WE LOAD PREVIOUSLY TRAINED MODELS
# LOAD_MODELS_FROM = None#'/kaggle/input/hms-efficientnetb0-pt-ckpts/'

# USE_KAGGLE_SPECTROGRAMS = True
# USE_EEG_SPECTROGRAMS = True

# df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
# TARGETS = df.columns[-6:]
# print('Train shape:', df.shape )
# print('Targets', list(TARGETS))


# train = df.groupby('eeg_id')[
#     ['spectrogram_id', 'spectrogram_label_offset_seconds']
# ].agg({'spectrogram_id': 'first', 'spectrogram_label_offset_seconds': 'min'})
# train.columns = ['spec_id', 'min']

# tmp = df.groupby('eeg_id')[
#     ['spectrogram_id','spectrogram_label_offset_seconds']
# ].agg({'spectrogram_label_offset_seconds' :'max'})
# train['max'] = tmp

# tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
# train['patient_id'] = tmp

# tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
# for t in TARGETS:
#     train[t] = tmp[t].values
    
# y_data = train[TARGETS].values
# y_data = y_data / y_data.sum(axis=1, keepdims=True)
# train[TARGETS] = y_data

# tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
# train['target'] = tmp

# train = train.reset_index()
# print('Train non-overlapp eeg_id shape:', train.shape )

# READ_SPEC_FILES = False

# # READ ALL SPECTROGRAMS
# PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
# files = os.listdir(PATH)
# print(f'There are {len(files)} spectrogram parquets')

# if READ_SPEC_FILES:    
#     spectrograms = {}
#     for i,f in enumerate(files):
#         if i % 100 == 0:
#             print(i, ', ', end='')
#         tmp = pd.read_parquet(f'{PATH}{f}')
#         name = int(f.split('.')[0])
#         spectrograms[name] = tmp.iloc[:,1:].values
# else:
#     spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()
    
    
# READ_EEG_SPEC_FILES = False

# if READ_EEG_SPEC_FILES:
#     all_eegs = {}
#     for i,e in enumerate(train.eeg_id.values):
#         if i % 100 == 0:
#             print(i, ', ', end='')
#         x = np.load(f'/kaggle/input/brain-eeg-spectrograms/EEG_Spectrograms/{e}.npy')
#         all_eegs[e] = x
# else:
#     all_eegs = np.load('/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()
    
    
# TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
# TARS2 = {x: y for y, x in TARS.items()}




In [ ]:
# import os
# import sys
# import gc
# sys.path.append('/kaggle/input/kaggle-kl-div')
# from kaggle_kl_div import score

# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
# import pytorch_lightning as pl
# import pandas as pd, numpy as np
# import matplotlib.pyplot as plt
# import albumentations as albu
# from sklearn.model_selection import KFold, GroupKFold
# class EEGDataset(Dataset):
    
#     def __init__(self, data, augment=False, mode='train', specs=spectrograms, eeg_specs=all_eegs): 
#         self.data = data
#         self.augment = augment
#         self.mode = mode
#         self.specs = specs
#         self.eeg_specs = eeg_specs
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         return self.__getitems__([index])
    
#     def __getitems__(self, indices):
#         X, y = self._generate_data(indices)
#         if self.augment:
#             X = self._augment(X) 
#         #if self.mode == 'train':
#         return list(zip(X, y))
#         #else:
#          #   return X
    
#     def _generate_data(self, indexes):
#         X = np.zeros((len(indexes), 128, 256, 8),dtype='float32')
#         y = np.zeros((len(indexes), 6),dtype='float32')
#         img = np.ones((128, 256),dtype='float32')
        
#         for j, i in enumerate(indexes):
#             row = self.data.iloc[i]
#             if self.mode == 'test': 
#                 r = 0
#             else: 
#                 r = int((row['min'] + row['max'])//4)

#             for k in range(4):
#                 # EXTRACT 300 ROWS OF SPECTROGRAM
#                 img = self.specs[row.spec_id][r:r+300, k*100:(k+1)*100].T
                
#                 # LOG TRANSFORM SPECTROGRAM
#                 img = np.clip(img, np.exp(-4), np.exp(8))
#                 img = np.log(img)
                
#                 # STANDARDIZE PER IMAGE
#                 ep = 1e-6
#                 m = np.nanmean(img.flatten())
#                 s = np.nanstd(img.flatten())
#                 img = (img - m) / (s + ep)
#                 img = np.nan_to_num(img, nan=0.0)
                
#                 # CROP TO 256 TIME STEPS
#                 X[j, 14:-14, :, k] = img[:, 22:-22] / 2.0
        
#             # EEG SPECTROGRAMS
#             img = self.eeg_specs[row.eeg_id]
#             X[j, :, :, 4:] = img
                
#             if self.mode != 'test':
#                 y[j,] = row[TARGETS]
            
#         return X, y
    
#     def _random_transform(self, img):
#         composition = albu.Compose([
#             albu.HorizontalFlip(p=0.5),
#             albu.CoarseDropout(max_holes=4,max_height=32,max_width=32,fill_value=0,p=0.5),
#             albu.RandomBrightnessContrast(p=0.25),
#             albu.transforms.CLAHE(p=.75),
#             albu.transforms.AdvancedBlur(p=.5),
#             albu.transforms.Sharpen(p=.25),
#         ])
#         return composition(image=img)['image']
            
#     def __augment(self, img_batch):
#         for i in range(img_batch.shape[0]):
#             img_batch[i,] = self._random_transform(img_batch[i,])
#         return img_batch
# # dataset = EEGDataset(train)
# # dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
# # ROWS = 2
# # COLS = 3
# # BATCHES = 2

# # for i, (x, y) in enumerate(dataloader):
# #     plt.figure(figsize=(20, 8))
# #     for j in range(ROWS):
# #         for k in range(COLS):
# #             plt.subplot(ROWS, COLS, j*COLS + k + 1)
# #             t = y[j*COLS + k]
# #             img = torch.flip(x[j*COLS+k, :, :, 0], (0,))
# #             mn = img.flatten().min()
# #             mx = img.flatten().max()
# #             img = (img-mn)/(mx-mn)
# #             plt.imshow(img)
# #             tars = f'[{t[0]:0.2f}]'
# #             for s in t[1:]:
# #                 tars += f', {s:0.2f}'
# #             eeg = train.eeg_id.values[i*32+j*COLS+k]
# #             plt.title(f'EEG = {eeg}\nTarget = {tars}',size=12)
# #             plt.yticks([])
# #             plt.ylabel('Frequencies (Hz)',size=14)
# #             plt.xlabel('Time (sec)',size=16)
# #     plt.show()
# #     if i == BATCHES-1:
# #         break
        
# #WEIGHTS_FILE = '/kaggle/input/hms-efficientnetb0-pt-ckpts/efficientnet_b0_rwightman-7f5810bc.pth'



In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

# CPMP: avoids some issues when using more than one worker
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#,global_pool='')
class EEGEffnetB0(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc0 = nn.Linear(8, 1)
        self.fc4 = nn.Linear(256, 1)
        #self.base_model = model_timm
        #self.base_model.load_state_dict(torch.load(WEIGHTS_FILE))
        #self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, 6, dtype=torch.float32)
        self.fc1 = nn.Linear(128, 4000)
        self.fc2 = nn.Linear(4000, 4000)
        self.fc3 = nn.Linear(4000, 6)
        
    def forward(self, x):
#         x1 = [x[:, :, :, i:i+1] for i in range(4)]
#         x1 = torch.concat(x1, dim=1)
#         x2 = [x[:, :, :, i+4:i+5] for i in range(4)]
#         x2 = torch.concat(x2, dim=1)
        
#         if USE_KAGGLE_SPECTROGRAMS & USE_EEG_SPECTROGRAMS:
#             x = torch.concat([x1, x2], dim=2)
#         elif USE_EEG_SPECTROGRAMS:
#             x = x2
#         else:
#             x = x1
#         x = torch.concat([x, x, x], dim=3)
        x = F.gelu(self.fc0(x))
        x = x.squeeze()
        x = F.gelu(self.fc4(x))
        x = x.squeeze()
        #x = x.permute(0, 3, 2,1)
        
        #x = F.gelu(self.base_model(x))
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        out = self.fc3(x)
        return F.log_softmax(out, dim=1)
    
kl_loss = nn.KLDivLoss(reduction='batchmean')
# from timm import create_model
# model_timm = create_model('twins_svt_base', pretrained=True, num_classes=2000)
all_oof = []
all_true = []
valid_loaders = []


    
device='cuda'
gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):  
    print('#'*25)
    print(f'### Fold {i+1}')
    bs=512
    train_ds = EEGDataset(train.iloc[train_index])
    train_loader = DataLoader(train_ds, shuffle=True, batch_size=bs, num_workers=4)
    valid_ds = EEGDataset(train.iloc[valid_index])
    valid_loader = DataLoader(valid_ds, shuffle=False, batch_size=bs, num_workers=4)
    
    print(f'### Train size: {len(train_index)}, Valid size: {len(valid_index)}')
    print('#'*25)
    
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader
    from torchvision import models
    #from your_dataset_file import EEGDataset  # Assume your EEGDataset class is saved in `your_dataset_file.py`
    from torch.optim.lr_scheduler import ExponentialLR



    # Optimizer (using SGD here, but you can choose others like Adam)


    # Exponential Learning Rate Scheduler
    # gamma is the factor by which the learning rate will be reduced at each epoch
    
    # Check if GPU is available and set the device accordingly
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Parameters
    learning_rate = 0.001
    num_epochs = 30


    # Define a simple model for demonstration purposes


    model = EEGEffnetB0().to(device)
    model=torch.nn.DataParallel(model, device_ids=[0, 1])
    valid_loaders.append(valid_loader)
    all_true.append(train.iloc[valid_index][TARGETS].values)

    # Loss and optimizer
    #criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = ExponentialLR(optimizer, gamma=0.995)

    # Training loop
    for epoch in range(num_epochs):
        running_loss=0
        model.train()
        
        for i, (X, y) in enumerate(train_loader):
            # Move tensors to the configured device
            X = X.to(device)
            y = y.to(device)

            # Forward pass
            outputs = model(X)
            loss = kl_loss(outputs, y)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            #if (i+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/i:.4f}')
        model.eval()
        running_loss=0
        with torch.inference_mode():
            for j, (X, y) in enumerate(valid_loader):
                # Move tensors to the configured device
                X = X.to(device)
                y = y.to(device)

                # Forward pass
                outputs = model(X)
                loss2 = kl_loss(outputs, y)

                # Backward and optimize
                running_loss += loss2.item()

            print(f' Valid Loss: {running_loss/len(valid_loader):.8f}')

    print("Training finished.")

    del trainer, model
    gc.collect()